# Circumferential Strain for Will's Toy Problem
## March 10, 2025

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import cheartio
import meshio
import pyvista as pv

In [37]:
data_source = "varying circumferential stiffness/brandon_stuff_bulge_refined"

xyz, ien, cell_type = cheartio.read_mesh(f"{data_source}/cyl")

cylindrical_coords0 = cheartio.read_dfile(f"{data_source}/Fiber-0.D")
cylindrical_coords100 = cheartio.read_dfile(f"{data_source}/Fiber-100.D")

unloaded_d = {"longitudinal_coord": cylindrical_coords0[:, 0:3],
              "circumferential_coord": cylindrical_coords0[:, 3:6],
              "radial_coord": cylindrical_coords0[:, 6:9],
              "space": cheartio.read_dfile(f"{data_source}/Space-0.D"),
              "stiffness": cheartio.read_dfile(f"{data_source}/Stiff-0.D"),
              "displacement": cheartio.read_dfile(f"{data_source}/Space-100.D") - cheartio.read_dfile(f"{data_source}/Space-0.D")}

loaded_d = {"longitudinal_coord": cylindrical_coords100[:, 0:3],
              "circumferential_coord": cylindrical_coords100[:, 3:6],
              "radial_coord": cylindrical_coords100[:, 6:9],
              "space": cheartio.read_dfile(f"{data_source}/Space-100.D"),
              "stiffness": cheartio.read_dfile(f"{data_source}/Stiff-0.D"),}


In [38]:
# NOTE: FINDING IDEAL CENTERLINE BY TAKING CENTROID EVERy 16 points (this is the number of points in the circumferential direction)
# Need to decide what kind of spline to fit?
def find_centerline(coords):
    centerline = []
    for i in range(0, len(coords), 32):
        centerline.append(np.mean(coords[i:i + 32], axis=0))
    return np.array(centerline)

In [47]:
print(unloaded_d["space"].shape)
print(32*65)

(2080, 3)
2080


In [48]:
centerline_unloaded = find_centerline(unloaded_d["space"])
centerline_loaded = find_centerline(loaded_d["space"])
displacement = centerline_loaded - centerline_unloaded

In [49]:
print(centerline_unloaded.shape)

(65, 3)


In [50]:
# Compute the gradient (finite difference) along the centerline for the loaded case
loaded_gradient = np.gradient(centerline_loaded, axis=0)
unloaded_gradient = np.gradient(centerline_unloaded, axis=0)

In [51]:
pairs = np.array([[2, i, i+1] for i in range(64)]) # PAIRS MAKE IT SO YOU CAN'T CALCULATE ARC LENGTH...
lines_idx = [centerline_loaded.shape[0]] + [i for i in range(centerline_loaded.shape[0])]
print(lines_idx)
#meshio.write_points_cells(f"{data_source}/centerline_unloaded.vtu",
#                          centerline_unloaded,
#                         lines,
#                          point_data={"Displacement": displacement,
#                                      "Derivative": unloaded_gradient,})
#meshio.write_points_cells(f"{data_source}/centerline_loaded.vtu",
#                            centerline_loaded,
#                             lines,
#                             point_data={"Derivative": loaded_gradient,})


centerline_unloaded = pv.PolyData(centerline_unloaded, lines=lines_idx)
centerline_unloaded["Displacement"] = displacement
centerline_unloaded["Derivative"] = unloaded_gradient
centerline_loaded = pv.PolyData(centerline_loaded, lines=lines_idx)
centerline_loaded["Derivative"] = loaded_gradient
centerline_unloaded.save(f"{data_source}/centerline_unloaded.vtp")
centerline_loaded.save(f"{data_source}/centerline_loaded.vtp")


[65, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]


In [35]:
cells = [("triangle", ien)]
meshio.write_points_cells(
    f"{data_source}/cyl0.vtu",
    xyz,
    cells,
    point_data={"longitudinal_coord": unloaded_d["longitudinal_coord"], 
                "circumferential_coord": unloaded_d["circumferential_coord"],
                "radial_coord": unloaded_d["radial_coord"],
                "stiffness": unloaded_d["stiffness"],
                "Displacement": unloaded_d["displacement"]},
)

meshio.write_points_cells(
    f"{data_source}/cyl100.vtu",
    loaded_d["space"],
    cells,
    point_data={"longitudinal_coord": loaded_d["longitudinal_coord"],
                "circumferential_coord": loaded_d["circumferential_coord"],
                "radial_coord": loaded_d["radial_coord"],
                "stiffness": unloaded_d["stiffness"]},
)

In [9]:
# Coord-0.D is a 9 dimension array, first 3 is the longitudinal direction unit vector, 
# second 3 is the circumferential direction unit vector, 
# and the last 3 is the unit vector for the radial direction

unloaded_d["radial_coord"].shape


(2080, 3)